In [1]:
# -*- coding: utf-8
# Abraji (https://www.abraji.org.br)
# Reinaldo Chaves (reinaldo@abraji.org.br)
# Acessa gastos de vereadores da Câmara Municipal de São Paulo
#

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Dica de Pedro Burgos: https://github.com/BurgosNY

Relatório Mensal por Natureza de Despesas da Câmara de Vereadores de SP

http://www.saopaulo.sp.leg.br/relatorio-por-natureza-de-despesa-partir-de-2015/

In [ ]:
# Exemplo de janeiro de 2019

In [ ]:
# No meu computador foi necessário fazer essas verificações de encoding e adaptações
# Isso depende de cada configuração e também do site que será raspado
# Talvez no seu computador não seja necessário

In [2]:
req = requests.get('https://sisgvarmazenamento.blob.core.windows.net/prd/PublicacaoPortal/Arquivos/201901.htm')
req.encoding

'ISO-8859-1'

In [3]:
# Quando você recebe uma resposta, Requests faz uma suposição sobre a codificação a ser usada para decodificar a resposta quando você acessa o atributo Response.text. 
# As solicitações primeiro verificarão uma codificação no cabeçalho HTTP e, se nenhuma estiver presente, usarão o chardet para tentar adivinhar a codificação.

# A única vez que Requests não fará isso é se nenhum charset explícito estiver presente nos cabeçalhos HTTP e o cabeçalho Content-Type contiver texto. 
# Nessa situação, o RFC 2616 especifica que o conjunto de caracteres padrão deve ser ISO-8859-1. 
# Requests segue a especificação neste caso. Se você precisar de uma codificação diferente, poderá definir manualmente a propriedade Response.encoding ou usar o Response.content bruto.

In [5]:
# Inspecionar os cabeçalhos de solicitação mostra que, de fato, "nenhum charset explícito está presente nos cabeçalhos HTTP e o cabeçalho Content-Type contém texto"

In [6]:
req.headers['content-type']

'text/html'

In [ ]:
# Portanto, o request segue fielmente o padrão e são decodificadas como ISO-8859-1 (latin-1).

# No conteúdo da resposta, um conjunto de caracteres é especificado

In [7]:
# No entanto, isso está errado: a decodificação como UTF-16 produz mojibake 
# Que é o texto truncado que é o resultado do texto ser decodificado usando uma codificação de caracteres não intencional

In [ ]:
# O chardet identifica corretamente a codificação como UTF-8.

In [8]:
#Então, para resumir:

# Não há uma maneira geral de determinar a codificação de texto com precisão total
# Nesse caso específico, a codificação correta é UTF-8.

In [9]:
# Então eu preciso trocar manualmente o encoding aqui

In [10]:
req.encoding = 'UTF-8'

In [11]:
sopa = BeautifulSoup(req.text,'lxml')

In [12]:
# Exemplo capta título principal

In [13]:
sopa.find('h1').text

'\r\n                        CÂMARA MUNICIPAL DE SÃO PAULO'

In [14]:
# Em Inspecionar elemento vemos que cada tabela de vereador fica em table e class=bloco

In [15]:
# Mostra a primeira tabela

In [16]:
sopa.find("table", {"class": "bloco"})

<table border="0" class="bloco" width="100%">
<tr>
<td>
<p class="itembusca"><b>
                        Vereador(a): ADILSON AMADEU</b></p>
</td>
</tr>
<tr>
<td>
<table align="center" border="1" width="100%">
<tr>
<td colspan="2">Natureza da despesa</td>
<td>Valor utilizado</td>
</tr>
<tr>
<td colspan="3">INTERMEDIADO - CORREIOS</td>
</tr>
<tr>
<td width="15%">50.176.288/0001-28</td>
<td width="60%">CAMARA MUNICIPAL DE SÃO PAULO</td>
<td align="RIGHT" width="20%">5.020,12</td>
</tr>
<tr>
<td align="RIGHT" colspan="2">TOTAL DO ITEM</td>
<td align="RIGHT">5.020,12</td>
</tr>
<tr>
<td colspan="3">INTERMEDIADO - LOCAÇÃO DE VEÍCULOS</td>
</tr>
<tr>
<td width="15%">50.176.288/0001-28</td>
<td width="60%">CAMARA MUNICIPAL DE SÃO PAULO</td>
<td align="RIGHT" width="20%">2.199,17</td>
</tr>
<tr>
<td align="RIGHT" colspan="2">TOTAL DO ITEM</td>
<td align="RIGHT">2.199,17</td>
</tr>
<tr>
<td align="RIGHT" colspan="2">TOTAL DO MÊS</td>
<td align="RIGHT">7.219,29</td>
</tr>
</table>
</td>
</tr>
<t

In [17]:
# Extrai todos os blocos

In [18]:
blocos = sopa.findAll("table", {"class": "bloco"})

Exemplos de como navegar

In [19]:
# O nome do quinto vereador

In [20]:
blocos[4].find("p").text.strip()

'Vereador(a): ANDRÉ SANTOS'

In [21]:
# Seu gasto total está na tag td

In [22]:
tds = blocos[4].find_all("td")

In [24]:
#tds

In [25]:
# E o valor final na antepenúltima td

In [26]:
tds[-3].text

'15.636,94'

In [27]:
# Local com o nome dos vereadores

In [28]:
primeiro_bloco = sopa.find("table", {"class": "bloco"})

In [29]:
vereador = primeiro_bloco.find("p").text.strip()
vereador

'Vereador(a): ADILSON AMADEU'

In [30]:
vereador = vereador.split(": ")[1]

In [31]:
vereador

'ADILSON AMADEU'

In [32]:
# Faz uma iteração pelos blocos e armazena resultados

In [33]:
planilha = [] # lista vazia armazenar os resultados

for linha in blocos:
    vereador = linha.find("p").text.strip()
    
    print(vereador)
    
    if vereador.startswith("Liderança"): # se o nome do vereador for Liderança não contabiliza
        break
    vereador = vereador.split(": ")[1] # pega o segundo valor a partir da divisão :
    vereador = vereador.title() # transforma em maiúscula
    
    celulas = linha.find_all("td") # valores
    valor = celulas[-3].text # valor total
    
    valor = valor.replace('.', '') # deixa no formato com decimal ,
    valor = valor.replace(',','.') # deixa no formato com decimal ,
    
    dicionario = {"vereador": vereador, "gasto": float(valor)} # cria dicionário com o resultado
    planilha.append(dicionario)

Vereador(a): ADILSON AMADEU
Vereador(a): ADRIANA RAMALHO
Vereador(a): ALESSANDRO GUEDES
Vereador(a): ALFREDINHO
Vereador(a): ANDRÉ SANTOS
Vereador(a): ARSELINO TATTO
Vereador(a): ATILIO FRANCISCO
Vereador(a): AURELIO NOMURA
Vereador(a): CAIO MIRANDA
Vereador(a): CAMILO CRISTÓFARO
Vereador(a): CELSO GIANNAZI
Vereador(a): CELSO JATENE
Vereador(a): CLAUDINHO DE SOUZA
Vereador(a): CLAUDIO FONSECA
Vereador(a): CONTE LOPES
Vereador(a): DALTON SILVANO
Vereador(a): DAVID SOARES
Vereador(a): DONATO
Vereador(a): EDIR SALES
Vereador(a): EDUARDO MATARAZZO SUPLICY
Vereador(a): EDUARDO TUMA
Vereador(a): ELISEU GABRIEL
Vereador(a): FABIO RIVA
Vereador(a): FERNANDO HOLIDAY
Vereador(a): GEORGE HATO
Vereador(a): GILBERTO NASCIMENTO
Vereador(a): GILSON BARRETO
Vereador(a): ISAC FELIX
Vereador(a): JAIR TATTO
Vereador(a): JANAINA LIMA
Vereador(a): JOSÉ POLICE NETO
Vereador(a): JULIANA CARDOSO
Vereador(a): MÁRIO COVAS NETO
Vereador(a): MILTON FERREIRA
Vereador(a): MILTON LEITE
Vereador(a): NATALINI
Vereador

In [34]:
# Transforma em dataframe

In [35]:
df = pd.DataFrame(planilha)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 2 columns):
gasto       56 non-null float64
vereador    56 non-null object
dtypes: float64(1), object(1)
memory usage: 976.0+ bytes


In [37]:
df.reset_index().head()

,index,gasto,vereador
0,0,7219.29,Adilson Amadeu
1,1,21111.66,Adriana Ramalho
2,2,20431.91,Alessandro Guedes
3,3,22911.42,Alfredinho
4,4,15636.94,André Santos


In [38]:
# Ordena gastos

In [39]:
df.sort_values(by="gasto", ascending=False)

,gasto,vereador
31,24871.88,Juliana Cardoso
33,24871.88,Milton Ferreira
27,24123.88,Isac Felix
15,24113.44,Dalton Silvano
19,23704.16,Eduardo Matarazzo Suplicy
55,23555.01,Zé Turin
3,22911.42,Alfredinho
18,22183.60,Edir Sales
17,21882.66,Donato
6,21877.55,Atilio Francisco


In [ ]:
df.to_csv('gastos_vereadores_sp_jan_2019.csv', index=False)

In [ ]:
df.to_excel('gastos_vereadores_sp_jan_2019.xlsx',sheet_name='Sheet1')